In [26]:
from bs4 import BeautifulSoup as bs
import requests
import re

In [2]:
my_url = 'https://www.espn.com/nfl/playbyplay?gameId=401030711'

In [102]:
def get_drive_info(game_ids):
    
    results = []
    
    for game in game_ids:
        
        my_url = 'https://www.espn.com/nfl/playbyplay?gameId={}'.format(game)

        team_dict = {}

        req = requests.get(my_url)
        my_text = req.text
        soup = bs(my_text, 'html.parser')

        headlines = [item.get_text() for item in soup.find_all('span', class_='headline')]

        sub_soup = soup.find(id='main-container')
        logos = sub_soup.find_all('span', class_='home-logo')

        team = re.compile("nfl/500/([a-z]{3})")

        team_abbr = []

        for stuff in logos:
            team_logo = team.findall(str(stuff))
            
            if team_logo:
                team_logo = team_logo[0]
            else:
                pass

            team_abbr.append(team_logo)

        detail_soup = sub_soup.find_all('span',class_='drive-details')

        yard_pattern = re.compile("(-?[0-9]*) yards?")
        yards = []
        for detail in detail_soup:
            deets = detail.get_text()
            yards.append(yard_pattern.findall(deets)[0])

        for i in range(len(headlines)):
            detail_tuple = (team_abbr[i],headlines[i],yards[i])
            results.append(detail_tuple)

            
    return results
        
        

In [103]:
def get_game_ids():
    game_ids = []
    
    my_url = 'https://www.espn.com/nfl/team/schedule/_/name/bal/season/2018'

    req = requests.get(my_url)
    my_text = req.text
    soup = bs(my_text, 'html.parser')
    
    sub_soup = soup.find('section',class_='pt0')
    
    links = sub_soup.find_all('a',class_='AnchorLink')
    
    gameID = re.compile("gameId/([0-9]*)")
    
    for stuff in links:
        stuff = str(stuff)
        
        game_id = gameID.findall(stuff)
        
        if game_id:
            game_ids.append(game_id[0])
    
    return game_ids


In [104]:
ids= get_game_ids()
get_drive_info(ids)

[('bal', 'Punt', '21'),
 ('lac', 'Punt', '7'),
 ('bal', 'Fumble', '-14'),
 ('lac', 'Field Goal', '12'),
 ('bal', 'Punt', '13'),
 ('lac', 'Field Goal', '7'),
 ('bal', 'Punt', '8'),
 ('lac', 'Punt', '29'),
 ('bal', 'Interception', '21'),
 ('lac', 'Field Goal', '37'),
 ('bal', 'Punt', '6'),
 ('lac', 'Field Goal', '58'),
 ('lac', 'Missed FG', '3'),
 ('bal', 'Punt', '-10'),
 ('lac', 'Fumble', '9'),
 ('bal', 'Field Goal', '6'),
 ('lac', 'Punt', '4'),
 ('bal', 'Missed FG', '8'),
 ('lac', 'Touchdown', '60'),
 ('bal', 'Punt', '-9'),
 ('lac', 'Field Goal', '16'),
 ('bal', 'Touchdown', '75'),
 ('lac', 'Punt', '5'),
 ('bal', 'Touchdown', '80'),
 ('lac', 'Punt', '3'),
 ('bal', 'Fumble', '-5'),
 ('lac', 'End of Game', '-1'),
 ('bal', 'Touchdown', '80'),
 ('buf', 'Punt', '-4'),
 ('bal', 'Punt', '1'),
 ('buf', 'Punt', '3'),
 ('bal', 'Touchdown', '66'),
 ('buf', 'Punt', '-3'),
 ('bal', 'Field Goal', '-3'),
 ('buf', 'Punt', '-4'),
 ('bal', 'Fumble', '5'),
 ('buf', 'Missed FG', '1'),
 ('bal', 'Punt', '-2